# Module 1: Multi-Vector Embeddings in Qdrant

In [1]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333")

In [2]:
client.create_collection(
    collection_name="colbert-tests",
    vectors_config={
        "colbert": models.VectorParams(
            # Size of an individual token vector
            size=128,
            # Distance function to be used for similarity
            distance=models.Distance.DOT,
            # Enable multivectors for that named vector
            multivector_config=models.MultiVectorConfig(
                # Use MaxSim comparison
                comparator=models.MultiVectorComparator.MAX_SIM,
            ),
            # Disable HNSW, as it won't be used either way
            hnsw_config=models.HnswConfigDiff(m=0),
        ),
    },
)

True

In [3]:
documents = [
    # Document A: Highly relevant - addresses all query aspects
    "When async tasks fail to return database connections, the pool "
    "becomes exhausted and requests start failing. Ensuring "
    "connections are closed after awaits prevents this.",

    # Document B: Partially relevant - mentions some concepts
    "Database resource exhaustion can occur due to limited pool sizes.",

    # Document C: Keyword-stuffed - contains related terms without substance
    "Understanding concurrency, async IO, and database performance in "
    "Python web applications.",

    # Document D: Completely irrelevant
    "Handling training for pythons should be done gradually, starting "
    "with short sessions and increasing duration as the snake becomes "
    "more comfortable.",
]

In [4]:
import uuid

client.upsert(
    collection_name="colbert-tests",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "colbert": models.Document(
                    text=doc,
                    model="colbert-ir/colbertv2.0",
                )
            },
            payload={
                "text": doc,
            }
        )
        for doc in documents
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [5]:
query = "How can I prevent Python database connection " \
        "pool exhaustion in async web applications?"

In [6]:
client.query_points(
    collection_name="colbert-tests",
    query=models.Document(
        text=query,
        model="colbert-ir/colbertv2.0",
    ),
    using="colbert",
    limit=2,
)

QueryResponse(points=[ScoredPoint(id='cade4005-d1f4-453d-8767-bc51c4ea51bb', version=0, score=21.181477, payload={'text': 'When async tasks fail to return database connections, the pool becomes exhausted and requests start failing. Ensuring connections are closed after awaits prevents this.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='6a2686ef-91d1-4e97-9b8b-b99bfbe651c7', version=0, score=16.222866, payload={'text': 'Database resource exhaustion can occur due to limited pool sizes.'}, vector=None, shard_key=None, order_value=None)])

## Storing vectors on disk

In [7]:
client.create_collection(
    collection_name="colbert-tests-on-disk",
    vectors_config={
        "colbert": models.VectorParams(
            size=128,
            distance=models.Distance.DOT,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM,
            ),
            hnsw_config=models.HnswConfigDiff(m=0),
            # Offload vectors to disk
            on_disk=True,
        ),
    },
)

True

In [8]:
client.upsert(
    collection_name="colbert-tests-on-disk",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "colbert": models.Document(
                    text=doc,
                    model="colbert-ir/colbertv2.0",
                )
            },
            payload={
                "text": doc,
            }
        )
        for doc in documents
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
client.query_points(
    collection_name="colbert-tests",
    query=models.Document(
        text=query,
        model="colbert-ir/colbertv2.0",
    ),
    using="colbert",
    limit=2,
)

QueryResponse(points=[ScoredPoint(id='cade4005-d1f4-453d-8767-bc51c4ea51bb', version=0, score=21.181477, payload={'text': 'When async tasks fail to return database connections, the pool becomes exhausted and requests start failing. Ensuring connections are closed after awaits prevents this.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='6a2686ef-91d1-4e97-9b8b-b99bfbe651c7', version=0, score=16.222866, payload={'text': 'Database resource exhaustion can occur due to limited pool sizes.'}, vector=None, shard_key=None, order_value=None)])